In [4]:
import nltk
from nltk.corpus import reuters
nltk.download('reuters')
from nltk.stem.porter import PorterStemmer
import string
import random
import numpy as np
from math import pow
from math import sqrt
from math import ceil
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

[nltk_data] Downloading package reuters to
[nltk_data]     /Users/sarthak.jain/nltk_data...


In [5]:
#Define our tokenizer
table = str.maketrans('', '', string.punctuation) #removes punctuation when used with translate
def tokenize(text):
    tokens = nltk.word_tokenize(text.lower().translate(table)) #lowe-case text, remove punctuation, tokenize
    stemmed_tokens = [PorterStemmer().stem(item) for item in tokens] #Porter Stemming
    return stemmed_tokens

In [6]:
#Preprocess documents
file_ids = reuters.fileids() #reuter file ids
original_doc_list = [reuters.raw(file_id) for file_id in file_ids] #original document content
mod_doc_list = [doc[(doc.index('\n') + 1):] for doc in original_doc_list] #document with title removed
no_of_docs = len(mod_doc_list)

In [7]:
#calculate idf value
vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words='english', use_idf=True)
corpus = mod_doc_list
vectorizer.fit(corpus)

//anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wo

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize at 0x1a15ec2620>, use_idf=True,
                vocabulary=None)

In [8]:
#create idf dictionary
features = vectorizer.get_feature_names()
idfarray = vectorizer.idf_
idf_dict = {}
for i in range(0, len(features)):
    idf_dict[features[i]] = idfarray[i]

In [9]:
#Return similrity matrix using idf-modified-cosine as similaity measure
def SimilarityMatrix(sent_text):
    no_of_sentences = len(sent_text)
    similarity_mat = np.zeros((no_of_sentences,no_of_sentences))  #initialized with zeroes
    for i in range(0, no_of_sentences):
        for j in range(0, no_of_sentences):
            if i == j:
                similarity_mat[i][j] = 1
            else:
                lis1 = tokenize(sent_text[i]) #token list for sentence i
                lis2 = tokenize(sent_text[j]) #token list for sentence j
                dict1 = Counter(lis1) #dictionary of term frequencies(term->tf)
                dict2 = Counter(lis2) #dictionary of term frequencies(term->tf)
                common_tokens = list(set(lis1 + lis2)) #common tokens in the 2 sentences

                numerator = 0
                for term in common_tokens:
                    if term in idf_dict:
                        numerator += dict1[term] * dict2[term] * idf_dict[term] * idf_dict[term]

                denom1 = 0
                denom2 = 0
                for token in lis1:
                    if token in idf_dict:
                        denom1 += pow(dict1[token] * idf_dict[token], 2)
                for token in lis2:
                    if token in idf_dict:
                        denom2 += pow(dict2[token] * idf_dict[token], 2)
                denominator = sqrt(denom1 * denom2)

                similarity_mat[i][j] = numerator / denominator #idf-modified-cosine

    return similarity_mat

In [10]:
#Apply page rank algorithm and return pagerank array based on the similarity matrix
def PageRank(similarity_mat):
    no_of_sentences = np.shape(similarity_mat)[0]
    #Calculate rowsum array i.e. the sum of similarity values for each sentence excluding the sentence itself
    rowsum_array = [sum(similarity_mat[i]) - similarity_mat[i][i] for i in range(0, no_of_sentences)]

    #Initialize pagerank array with value (1/no_of_sentences) for each sentence
    pagerank_array = [(1 / no_of_sentences)] * no_of_sentences

    iterations_threshold = 100000 #No of consecutive iterations for stopping
    pages_threshold = no_of_sentences #Threshold on the no of pages to be counted as a stable iteration
    percentage_threshold = pow(10, -250) #Threshold on the percentage to be counted as a stable page
    no_of_iterations = 200000 #No of total iterations after which to terminate the loop

    consecutive_iterations = 0 #No of consecutive stable iterations at any point
    iterations_count = 0 #Total iterations till now

    while consecutive_iterations < iterations_threshold and iterations_count < no_of_iterations:
        #Caclulate new pageranks at (i+1)th iteration
        mod_pagerank = []
        for i in range(0, no_of_sentences):
            new_pagerank_value = 0.85 / no_of_sentences
            idf_similarity_sum = 0
            for j in range(0, no_of_sentences):
                if i != j and rowsum_array[j] != 0:
                    idf_similarity_sum += (similarity_mat[i][j] * pagerank_array[j]) / rowsum_array[j]
            new_pagerank_value += 0.15 * idf_similarity_sum
            mod_pagerank.append(new_pagerank_value)

        no_of_positives = 0 #No of pages having change less than the threshold in the current iteration
        for i in range(0, len(pagerank_array)):
            abs_difference = abs(mod_pagerank[i] - pagerank_array[i])
            if abs_difference <= (percentage_threshold * pagerank_array[i]):
                no_of_positives += 1
        if no_of_positives >= ceil(pages_threshold):
            consecutive_iterations += 1
        else:
            consecutive_iterations = 0

        iterations_count += 1
        pagerank_array = mod_pagerank
        
    return pagerank_array

In [12]:
#Select a random document 
doc_no = random.randint(0, no_of_docs)
#doc_no=8425
text = mod_doc_list[doc_no]
sent_text = nltk.sent_tokenize(text)
similarity_matrix = SimilarityMatrix(sent_text)
pagerank_array = PageRank(similarity_matrix)
print("The document is:")
print(text)


original_title=''
complete_text=reuters.raw(file_ids[doc_no])

j=0
while(complete_text[j] != '\n'):
        original_title=original_title+complete_text[j]
        j = j + 1
print("The original title is:")
print(original_title)
print()


print("The title according to lexrank is:")
print(sent_text[pagerank_array.index(max(pagerank_array))])    

The document is:
  Algeria will tender on April 3 for
  20,000 tonnes of optional origin sunflowerseed oil/rapeseed oil
  for Apr/May loading, traders said.
      Meanwhile, the market is awaiting results of an Algerian
  import tender which took place over the weekend for about
  10,000 tonnes of refined vegetable oils in drums, traders
  added.
  


The original title is:
ALGERIA SETS TENDER FOR RAPE/SUNFLOWERSEED OIL

The title according to lexrank is:
  Algeria will tender on April 3 for
  20,000 tonnes of optional origin sunflowerseed oil/rapeseed oil
  for Apr/May loading, traders said.
